<a href="https://colab.research.google.com/github/nwang2208/Bioinformatics-Class-Projects/blob/mapping-reads/Project2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Bio

In [ ]:
from Bio import SeqIO
import math
import numpy as np

Opening Bound Reads

In [ ]:
bound_dict = {}
for seq_record in SeqIO.parse('/content/drive/MyDrive/122data/PROJECT2B/bound.fasta', "fasta"):
    bound_dict[seq_record.id] = str(seq_record.seq)

Make a reverse complement set of bound reads (did not include in final answer)

In [ ]:
r_bound_dict= {}

for key,value in bound_dict.items():
  r_seq = ""
  for x in range(0,len(value)):
    base = ''
    if(value[x] == "A"):
      base = 'T'
    elif(value[x] == "T"):
      base = 'A'
    elif(value[x] == "C"):
      base = 'G'
    else:
      base = 'C'
    r_seq += base
  r_bound_dict[key] = r_seq[::-1]

In [ ]:
r_bound_dict

Building PWM

In [ ]:
def profileMatrix(kmer_list, k):
    countA = 1 #pseudocounts
    countC = 1
    countG = 1
    countT = 1
    col_list = []

    for i in range(0, k):
        for el in kmer_list:
            if(el[i] == 'A'):
                countA += 1
            elif(el[i] == 'C'):
                countC += 1
            elif(el[i] == 'G'):
                countG += 1
            else:
                countT += 1
        sum = countA + countC + countG + countT
        col = [countA/sum, countC/sum, countG/sum, countT/sum]
        col_list.append(col)
        col = []
        countA = 1
        countC = 1
        countG = 1
        countT = 1

    return(col_list)

In [ ]:
def score(profile_matrix, motif_list):
    score_list = []
    for el in motif_list:
        score = 1
        for i in range(0, len(el)):

            if(el[i] == 'A'):
                score *= profile_matrix[i][0]
            elif(el[i] == 'C'):
                score *= profile_matrix[i][1]
            elif(el[i] == 'G'):
                score *= profile_matrix[i][2]
            else:
                score *= profile_matrix[i][3]
        score_list.append(score)
    return(score_list)


In [ ]:
def Score(Motifs):
    score = 0
    for i in range(len(Motifs[0])):
        j = [motif[i] for motif in Motifs]
        score += (len(j) - max(j.count("A"), j.count("C"), j.count("T"), j.count("G")))
    return score

In [ ]:
def GreedyMotifSearch(dna, k, t):
    bestMotifs = []
    for el in dna:
        seq = el[0:k]
        bestMotifs.append(seq)

    bestProfile = profileMatrix(bestMotifs, k)

    motif_list = []
    temp_list = []
    for i in range(0, len(dna[0])-k+1):
        motif1 = dna[0][i:i+k]
        motif_list.append(motif1)
        profile = profileMatrix(motif_list, k)



        for j in range(1, t): #look at the other strings
            for x in range(0, len(dna[j])-k+1):
                motif = dna[j][x:x+k]
                temp_list.append(motif)
            score_list = score(profile, temp_list)
            indexofbest = score_list.index(max(score_list))
            best_motif = temp_list[indexofbest]
            motif_list.append(best_motif)
            profile = []
            profile = profileMatrix(motif_list, k)
            temp_list = []


        if(Score(motif_list) < Score(bestMotifs)):
            bestMotifs = []
            bestProfile = []
            bestMotifs = motif_list.copy()
            bestProfile = profile.copy()

        motif_list = []
        profile = []
        score_list1 = []
        score_list2 = []



    return(bestMotifs, bestProfile)

In [ ]:
k = 21
t = len(bound_dict)

dna = []
for key, value in bound_dict.items():
    dna.append(value)
'''
for key, value in r_bound_dict.items():
    dna.append(value)'''



In [ ]:
len(bound_dict)

In [ ]:
bestProfile = GreedyMotifSearch(dna, k, t)[1]

In [ ]:
bestProfile

In [ ]:
np.savetxt("pwm2.csv",
           bestProfile,
           delimiter =" ",  # Set the delimiter as a comma followed by a space
           fmt ='% s')